In [1]:
from gurobipy import *
import pandas as pd
import numpy as np
import csv

In [182]:
df = pd.read_excel("../data/焚化廠資料.xlsx")
df

,焚化廠名稱,縣市,年份,其他成本,勞動成本,焚化處理量,設計處理量,設計發電量,實際發電量,二氧化碳當量
0,宜蘭縣利澤垃圾資源回收(焚化)廠,宜蘭縣,2020,960330.0,1033323.0,182263.4,600,14700000,99631.87,136697.550
1,宜蘭縣利澤垃圾資源回收(焚化)廠,宜蘭縣,2011,329010.0,468548.0,199732.2,600,14700000,107838.70,149799.150
2,宜蘭縣利澤垃圾資源回收(焚化)廠,宜蘭縣,2019,846509.0,999635.0,202111.2,600,14700000,107890.40,151583.400
3,宜蘭縣利澤垃圾資源回收(焚化)廠,宜蘭縣,2012,371861.0,518953.0,202728.5,600,14700000,110156.70,152046.375
4,宜蘭縣利澤垃圾資源回收(焚化)廠,宜蘭縣,2013,411821.0,587162.0,205014.9,600,14700000,110901.60,153761.175
...,...,...,...,...,...,...,...,...,...,...
235,臺南市城西垃圾焚化廠,臺南市,2014,1085896.0,1285597.5,219334.7,900,14300000,91008.24,164501.025
236,臺南市城西垃圾焚化廠,臺南市,2016,1196041.0,1387306.0,219850.9,900,14300000,92339.52,164888.175
237,臺南市城西垃圾焚化廠,臺南市,2015,1096999.5,1299537.5,221728.8,900,14300000,93911.04,166296.600
238,臺南市城西垃圾焚化廠,臺南市,2011,835958.0,999123.0,222657.9,900,14300000,91025.38,166993.425


In [183]:
df.describe()

,年份,其他成本,勞動成本,焚化處理量,設計處理量,設計發電量,實際發電量,二氧化碳當量
count,240.000000,2.400000e+02,2.400000e+02,240.000000,240.000000,2.400000e+02,240.000000,240.000000
mean,2015.500000,1.153994e+06,1.564774e+06,264436.511708,1027.083333,2.311321e+07,133369.255292,198327.383781
std,2.878284,7.265852e+05,8.473039e+05,90906.892564,371.860404,1.154901e+07,62467.998158,68180.169423
min,2011.000000,9.946700e+04,4.506970e+05,63239.450000,300.000000,2.326000e+06,15989.940000,47429.587500
25%,2013.000000,7.192419e+05,1.009241e+06,199559.525000,900.000000,1.452500e+07,90917.182500,149669.643750
50%,2015.500000,9.426180e+05,1.307989e+06,253034.050000,900.000000,2.255000e+07,132502.500000,189775.537500
75%,2018.000000,1.750400e+06,2.206284e+06,303204.325000,1350.000000,2.790000e+07,170481.750000,227403.243750
max,2020.000000,3.730386e+06,5.417133e+06,472580.700000,1800.000000,4.900000e+07,270666.500000,354435.525000


In [186]:
import random 

#df.groupby(df.年份).get_group(2020)
df["二氧化碳當量"] = df["二氧化碳當量"].apply(lambda x: x * random.uniform(0.9,1.1))

In [187]:
df.describe()

,年份,其他成本,勞動成本,焚化處理量,設計處理量,設計發電量,實際發電量,二氧化碳當量
count,240.000000,2.400000e+02,2.400000e+02,240.000000,240.000000,2.400000e+02,240.000000,240.000000
mean,2015.500000,1.153994e+06,1.564774e+06,264436.511708,1027.083333,2.311321e+07,133369.255292,200713.251217
std,2.878284,7.265852e+05,8.473039e+05,90906.892564,371.860404,1.154901e+07,62467.998158,70147.549918
min,2011.000000,9.946700e+04,4.506970e+05,63239.450000,300.000000,2.326000e+06,15989.940000,47711.364648
25%,2013.000000,7.192419e+05,1.009241e+06,199559.525000,900.000000,1.452500e+07,90917.182500,150526.442533
50%,2015.500000,9.426180e+05,1.307989e+06,253034.050000,900.000000,2.255000e+07,132502.500000,195648.418723
75%,2018.000000,1.750400e+06,2.206284e+06,303204.325000,1350.000000,2.790000e+07,170481.750000,237659.439374
max,2020.000000,3.730386e+06,5.417133e+06,472580.700000,1800.000000,4.900000e+07,270666.500000,377122.204573


In [188]:
# Sets
Year = list(range(2011, 2021))
K = list(range(24))  # number of DMUs
I_V = [0,1,2]  # variable input amount
I_F = [0,1]  # fixed input amount
J = [0]  # output 
M = [0]  # bad output
years = list(range(1,10))
dmus = list(range(1,25))

In [189]:
year = 2020
# Parameters
X_V = [
    list(df[df["年份"] == year]["其他成本"].values),
    list(df[df["年份"] == year]["勞動成本"].values),
    list(df[df["年份"] == year]["焚化處理量"].values)
]
X_F = [
    list(df[df["年份"] == year]["設計處理量"].values),
    list(df[df["年份"] == year]["設計發電量"].values)
]
Y = [list(df[df["年份"] == year]["實際發電量"].values)]
B = [list(df[df["年份"] == year]["二氧化碳當量"].values)]

In [173]:
sum(B[0]) * 0.01

44775.071978051135

In [215]:
NT = 0
lambda_NT = [[] for r in K]
omega_NT = [[] for r in K]

md11 = Model('model')
_lambda = md11.addVars(K, K, vtype=GRB.CONTINUOUS, name='lambda')
_omega = md11.addVars(K, K, vtype=GRB.CONTINUOUS, name='omega')
_theta = md11.addVars(K, vtype=GRB.CONTINUOUS, name='theta') 
c = md11.addVar(vtype=GRB.CONTINUOUS, name='c') 

md11.setObjective(
    # quicksum(_theta[r] * Y[j][r] for r in K for j in J), GRB.MAXIMIZE
    c, GRB.MINIMIZE

)

for r in K:
    md11.addConstrs(
        quicksum(_lambda[k,r] * X_F[i][k] for k in K) <= X_F[i][r]
        for i in I_F
    )

    md11.addConstrs(
        quicksum(_lambda[k,r] * X_V[i][k] for k in K) <= X_V[i][r]
        for i in I_V
    )

    md11.addConstrs(
        quicksum(_lambda[k,r] * Y[j][k] for k in K) >= _theta[r] * Y[j][r]
        for j in J
    )

    md11.addConstr(
        quicksum(_lambda[k,r] for k in K) == 1
    )

    md11.addConstr(
        quicksum(_omega[k,r] * X_V[2][k] for k in K) >=  X_V[2][r]
    )

    md11.addConstrs(
        quicksum(_omega[k,r] * B[m][k] for k in K) <=  B[m][r]
        for m in M
    )

    md11.addConstr(
        quicksum(_omega[k,r] for k in K) == 1
    )

    md11.addConstr(
        quicksum(_omega[k,r] * X_V[2][k] for k in K) == quicksum(_lambda[k,r] * X_V[2][k] for k in K)
    )

md11.addConstr(
    quicksum(_omega[k,r] * B[0][k] for k in K for r in K) <= sum(B[0]) * c
)

# md11.params.LogToConsole = 0
# md11.params.NonConvex = 2
md11.optimize() 

print(df.loc[r,"焚化廠名稱"])
for var in md11.getVars():
    print(f"{var.VarName}, {var.X}")

lambda_list = md11.getAttr('X', _lambda)
omega_list = md11.getAttr('X', _omega)

for r in K:
    for k in K:
        lambda_CIT[r].append(lambda_list[r, k])
        omega_CIT[r].append(omega_list[r, k])

# to load crs
NT = md11.objVal

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 265 rows, 1177 columns and 7513 nonzeros
Model fingerprint: 0xa7fe65e8
Coefficient statistics:
  Matrix range     [1e+00, 5e+07]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+07]
Presolve removed 25 rows and 25 columns
Presolve time: 0.01s
Presolved: 240 rows, 1152 columns, 6336 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.6692315e-01   1.113265e+04   0.000000e+00      0s
     171    9.1261561e-01   0.000000e+00   0.000000e+00      0s

Solved in 171 iterations and 0.05 seconds (0.00 work units)
Optimal objective  9.126156142e-01
苗栗縣垃圾焚化廠
lambda[0,0], 1.0
lambda[0,1], 0.6490146882119798
lambda[0,2], 0.0
lambda[0,3], 0.0
lambda[0,4], 0.0
lambda[0,5], 0.1554935723352757
lambda[0,6], 0.592953375639414
lambda[0,7], 0.00789159778096455
la

In [216]:
NT

0.9126156141973266

In [192]:
writer = pd.ExcelWriter('../results/NT.xlsx')

pd.DataFrame(lambda_NT).to_excel(writer, sheet_name='lambda')
pd.DataFrame(omega_NT).to_excel(writer, sheet_name='omega')
writer.save()

In [210]:
CIT = 0
lambda_CIT = [[] for r in K]
omega_CIT = [[] for r in K]
delta_CIT = [[] for r in K]

md11 = Model('model')
_lambda = md11.addVars(K, K, vtype=GRB.CONTINUOUS, name='lambda')
_omega = md11.addVars(K, K, vtype=GRB.CONTINUOUS, name='omega')
_theta = md11.addVars(K, vtype=GRB.CONTINUOUS, name='theta') 
_delta = md11.addVars(K, vtype=GRB.CONTINUOUS, name='delta') 
c = md11.addVar(vtype=GRB.CONTINUOUS, name='c') 

md11.setObjective(
    quicksum(_theta[r] * Y[j][r] for r in K for j in J), GRB.MAXIMIZE
)

for r in K:
    md11.addConstrs(
        quicksum(_lambda[k,r] * X_F[i][k] for k in K) <= X_F[i][r]
        for i in I_F
    )

    md11.addConstrs(
        quicksum(_lambda[k,r] * X_V[i][k] for k in K) <= X_V[i][r]
        for i in I_V
    )

    md11.addConstrs(
        quicksum(_lambda[k,r] * Y[j][k] for k in K) >= _theta[r] * Y[j][r]
        for j in J
    )

    md11.addConstr(
        quicksum(_lambda[k,r] for k in K) == 1
    )

    md11.addConstr(
        quicksum(_omega[k,r] * X_V[2][k] for k in K) >=  X_V[2][r]
    )

    md11.addConstrs(
        quicksum(_omega[k,r] * B[m][k] for k in K) <=  _delta[r] * B[m][r]
        for m in M
    )

    md11.addConstr(
        quicksum(_omega[k,r] for k in K) == 1
    )

    md11.addConstr(
        quicksum(_omega[k,r] * X_V[2][k] for k in K) ==  quicksum(_lambda[k,r] * X_V[2][k] for k in K)
    )

md11.addConstr(
    quicksum(_delta[k] * B[0][k] for k in K) <=  sum(B[0])
)

# md11.params.LogToConsole = 0
md11.params.NonConvex = 2
md11.optimize() 

print(df.loc[r,"焚化廠名稱"])
for var in md11.getVars():
    print(f"{var.VarName}, {var.X}")

lambda_list = md11.getAttr('X', _lambda)
omega_list = md11.getAttr('X', _omega)
delta_list = md11.getAttr('X', _delta)

delta_CIT = delta_list
for r in K:
    for k in K:
        lambda_CIT[r].append(lambda_list[r, k])
        omega_CIT[r].append(omega_list[r, k])        


# to load crs
CIT = md11.objVal

Set parameter NonConvex to value 2
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 265 rows, 1201 columns and 6985 nonzeros
Model fingerprint: 0x8612b3d1
Coefficient statistics:
  Matrix range     [1e+00, 5e+07]
  Objective range  [1e+00, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+07]
Presolve removed 49 rows and 49 columns
Presolve time: 0.04s
Presolved: 216 rows, 1152 columns, 5760 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.6412069e+06   4.137493e+04   0.000000e+00      0s
     153    3.4096902e+06   0.000000e+00   0.000000e+00      0s

Solved in 153 iterations and 0.06 seconds (0.00 work units)
Optimal objective  3.409690219e+06
苗栗縣垃圾焚化廠
lambda[0,0], 1.0
lambda[0,1], 0.0
lambda[0,2], 0.0
lambda[0,3], 0.0
lambda[0,4], 0.0
lambda[0,5], 0.0
lambda[0,6], 0.0
lambda[0,7], 0.0
lambda[0,8], 0.0
lambda[0,9

In [211]:
CIT

3409690.218533572

In [212]:
delta_CIT

{0: 0.851006981357894,
 1: 0.8495543295615986,
 2: 0.8879343900415682,
 3: 1.0,
 4: 0.9402156783294443,
 5: 0.9309033504596563,
 6: 0.8791071910114662,
 7: 0.8927671032694174,
 8: 0.9992322945892975,
 9: 1.0,
 10: 0.8926281157625301,
 11: 0.884211280331646,
 12: 0.8573924108928358,
 13: 1.0,
 14: 0.862191495199957,
 15: 0.8393196574882894,
 16: 0.8509091559652812,
 17: 0.9579204340632632,
 18: 1.0,
 19: 0.8835650387029395,
 20: 0.8787161119751484,
 21: 0.9476733075050027,
 22: 0.8576109375634016,
 23: 0.9143322388490971}

In [196]:
writer = pd.ExcelWriter('../results/CIT.xlsx')

pd.DataFrame(lambda_CIT).T.to_excel(writer, sheet_name='lambda')
pd.DataFrame(omega_CIT).T.to_excel(writer, sheet_name='omega')
writer.save()